In [4]:
# loading libraries
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader

In [32]:
# loading data
titanic_raw = pd.read_csv('titanic.csv')

#Processing data

titanic_processed = pd.DataFrame()

titanic_processed['Survived'] = titanic_raw['Survived']
titanic_processed['pclass'] = titanic_raw['Pclass']
titanic_processed['Sex'] = (titanic_raw['Sex'] == "male").astype(int)
titanic_processed['SibSp'] = titanic_raw['SibSp']
titanic_processed['Parch'] = titanic_raw['Parch']
titanic_processed['Age'] = titanic_raw['Age'].fillna(titanic_raw['Age'].mean())

# Splitting data
titanic_train = titanic_processed.sample(frac=0.8, random_state=200)
titanic_test = titanic_processed.drop(titanic_train.index)

x_train = titanic_train.drop('Survived', axis=1).values.astype(np.float32)
y_train = titanic_train['Survived'].values.astype(np.float32)

In [35]:
# defining dataloader

class MyDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.from_numpy(x_data).float()
        self.y_data = torch.from_numpy(y_data).long()
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

# creating the model

class Layer(torch.nn.Module):
    """A simple feedforward layer with an activation function."""
    def __init__(self, n_feature, n_hidden, activation_function=None):
        super().__init__()

        self.linear = torch.nn.Linear(n_feature, n_hidden)

        if activation_function is None:
            self.activation_function = torch.nn.Sigmoid()
        else:
            self.activation_function = activation_function

    def forward(self, x):
        out = self.linear(x)
        out = self.activation_function(out)
        return out


class FeedForward(torch.nn.Module):
    """A simple feedforward neural network with two hidden layers."""
    def __init__(self, n_feature, n_hidden, n_output, activation_function, activation_function_last):
        super().__init__()

        self.layer0 = Layer(n_feature=n_feature, n_hidden=n_hidden, activation_function=activation_function)
        self.layer1 = Layer(n_feature=n_hidden, n_hidden=n_hidden, activation_function=activation_function)
        self.layer2 = Layer(n_feature=n_hidden, n_hidden=n_output, activation_function=activation_function_last)

    def forward(self, x):
        # (b, f) -> (b, h)
        out = self.layer0(x)
        # (b, h) -> (b, h)
        out = self.layer1(out)
        # (b, h) -> (b, o)
        out = self.layer2(out)
        return out

In [36]:
# initiating dataloader
dataset = MyDataset(x_train, y_train)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [37]:
# Create a loss function
loss_fn = torch.nn.BCELoss()

# Create a feedforward neural network
feedforward = FeedForward(n_feature=5, n_hidden=64, n_output=1, activation_function=torch.nn.ReLU(), activation_function_last=torch.nn.Sigmoid()).cuda()

# Create an optimizer
optimizer = torch.optim.AdamW(feedforward.parameters(), lr=0.001, weight_decay=0.0)

In [40]:
# Training the model

for epoch in range(100):
    
    i = 0
    for x_batch, y_batch in dataloader:
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = feedforward(x_batch.cuda())

        # Compute and print loss
        loss = loss_fn(y_pred, y_batch.unsqueeze(1).float().cuda())
        print(f'Epoch {epoch + 1} | Batch {i + 1} | Loss: {loss.item()}')

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1

# Testing the model
        

Epoch 1 | Batch 1 | Loss: 0.29383862018585205
Epoch 1 | Batch 2 | Loss: 0.3605947196483612
Epoch 1 | Batch 3 | Loss: 0.16482847929000854
Epoch 1 | Batch 4 | Loss: 0.5793150067329407
Epoch 1 | Batch 5 | Loss: 0.222622811794281
Epoch 1 | Batch 6 | Loss: 0.1884271800518036
Epoch 1 | Batch 7 | Loss: 0.5221000909805298
Epoch 1 | Batch 8 | Loss: 0.2671268582344055
Epoch 1 | Batch 9 | Loss: 0.3048456907272339
Epoch 1 | Batch 10 | Loss: 0.5738648176193237
Epoch 1 | Batch 11 | Loss: 0.25158417224884033
Epoch 1 | Batch 12 | Loss: 0.49690428376197815
Epoch 1 | Batch 13 | Loss: 0.11133798211812973
Epoch 1 | Batch 14 | Loss: 0.1097010001540184
Epoch 1 | Batch 15 | Loss: 0.16911830008029938
Epoch 1 | Batch 16 | Loss: 0.20409291982650757
Epoch 1 | Batch 17 | Loss: 0.4399993121623993
Epoch 1 | Batch 18 | Loss: 0.35786980390548706
Epoch 1 | Batch 19 | Loss: 0.2173219472169876
Epoch 1 | Batch 20 | Loss: 0.4424092173576355
Epoch 1 | Batch 21 | Loss: 0.35384297370910645
Epoch 1 | Batch 22 | Loss: 0.434243